# Multi-Agent Orchestration

In this notebook, you'll create a team of agents that can create detailed research reports.

Here's how this will work:

1. User ask question to lead agent.
2. Lead agent (LA) decomposes question and creates report outline.
3. Lead agent calls research agent (RA) with instructions for section 1.
4. RA examines instructions and calls `search_pmc` and `gather_evidence` tools multiple times. Every time `gather_evidence` finishes, it saves an evidence record to DynamoDB.
6. RA returns a research summary and source information to LA.
7. LA updates outline with information from RA.
8. Repeat steps 3-7 for all outline sections.
9. Once research is complete, LA submits outline to writing agent (WA).
10. WA retrieves evidence records from DynamoDB and generates report text with inline citations using Claude citation API.
11. WA reports completion to LA
12. LA shares final report with user.

## 1. Prerequisites

- Python 3.12 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude 3.7 Sonnet model in Amazon Bedrock
- Basic understanding of Python programming

In [ ]:
%pip install -U -r requirements.txt

In [1]:
MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"

## 2. Define PMC Research Agent

This agent will be similar to the gather evidence agent we created in notebook 2, but with a change in how the evidence is stored. To avoid the "game of telephone" problem, we are going to deterministically store the gathered evidence in a DynamoDB table. This is a big advantage to building AI agents on AWS - they can leverage any of the 200+ services to store and process data.

Let's test out the updated tool to see how it works.

First, we create a DynamoDB table, with `toolUseId` as the primary key and `pmcid` as a global secondary id. This gives us the flexibility to extract evidence records either by tool use or paper. 

In [ ]:
import boto3

dynamodb = boto3.resource("dynamodb")

# Check if table exists
table_name = "deep-research-evidence"
try:
    table = dynamodb.Table(table_name)
    table.load()
    print(f"Table '{table_name}' already exists")
except:
    # Create table if it doesn't exist
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[{"AttributeName": "evidence_id", "KeyType": "HASH"}],
        AttributeDefinitions=[
            {"AttributeName": "evidence_id", "AttributeType": "S"},
            {"AttributeName": "pmc_id", "AttributeType": "S"},
        ],
        GlobalSecondaryIndexes=[
            {
                "IndexName": "pmc_id_index",
                "KeySchema": [{"AttributeName": "pmc_id", "KeyType": "HASH"}],
                "Projection": {"ProjectionType": "ALL"},
            }
        ],
        BillingMode="PAY_PER_REQUEST",
    )

    table.wait_until_exists()

Next, we directly invoke the updated `gather_evidence` tool and pass the db table name as an environment variable.

In [3]:
from strands import Agent
from search_pmc import search_pmc_tool
from gather_evidence_ddb import gather_evidence_tool
import os

os.environ["EVIDENCE_TABLE_NAME"] = table_name

MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"
agent = Agent(tools=[gather_evidence_tool], model=MODEL_ID)

# Send a message to the agent
agent.tool.gather_evidence_tool(
    pmc_id="PMC9438179",
    question="How safe and effective are GLP-1 drugs for long term use?",
)

INFO | gather_evidence | Starting gather_evidence for PMCID: PMC9438179, question: How safe and effective are GLP-1 drugs for long term use?
INFO | gather_evidence | Attempting to download s3://pmc-oa-opendata/oa_comm/txt/all/PMC9438179.txt to my_papers/PMC9438179/txt/PMC9438179.txt
INFO | gather_evidence | Successfully downloaded file to my_papers/PMC9438179/txt/PMC9438179.txt
INFO | gather_evidence | Successfully retrieved commercial article PMC9438179
INFO | gather_evidence | Processing paper with paper-qa for question: How safe and effective are GLP-1 drugs for long term use?
INFO | gather_evidence | Invoking paper-qa
INFO | gather_evidence | Successfully answered question for PMC9438179
INFO | gather_evidence | Saving record to deep-research-evidence
INFO | gather_evidence | {'evidence_id': '73c5ecc3-f3d2-4134-bc9a-28d7316ad4ff', 'question': 'How safe and effective are GLP-1 drugs for long term use?', 'answer': "Based on the available evidence, tirzepatide, a dual GLP-1/GIP recept

{'status': 'success',
 'content': [{'text': "Based on the available evidence, tirzepatide, a dual GLP-1/GIP receptor co-agonist, demonstrates favorable long-term safety and efficacy profiles. The Phase-3 SURPASS trials show sustained HbA1c reduction and weight loss over 52 weeks of treatment (Nauck2022 chunk 2). \n\nRegarding safety, tirzepatide exhibits a safety profile comparable to established GLP-1 receptor agonists, with no major cardiovascular safety signals identified during the 52-week study period (Nauck2022 chunk 2). This suggests that the dual receptor mechanism does not introduce additional cardiovascular risks beyond those associated with traditional GLP-1 therapies.\n\nFor efficacy, the sustained reductions in both glycemic control (HbA1c) and body weight over the full 52-week treatment period indicate maintained therapeutic benefits without significant attenuation of effect (Nauck2022 chunk 2). The long-term tolerability profile supports continued use without major disco

In [ ]:
# from strands import Agent
# from search_pmc import search_pmc_tool
# from gather_evidence_ddb import gather_evidence_tool
# import os

# MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"
# QUERY = "How safe and effective are GLP-1 drugs for long term use?"

# SYSTEM_PROMPT = """You are a life science research assistant. When given a scientific question, follow this process:

# 1. Use search_pmc_tool with max_search_result_count between 200 and 500 and max_filtered_result_count between 10 and 20 to find highly-cited papers. Search broadly first, then narrow down. Use temporal filters like "last 5 years"[dp] for recent work. 
# 2. Identify the PMC ID value for the most relevant paper, then submit the ID and the query to the gather_evidence_tool.
# 3. Generate a concise answer to the question based on the most relevant evidence, along with PMC ID and URL citations
# """

# os.environ['EVIDENCE_TABLE_NAME'] = table_name


# # Initialize your agent
# pmc_research_agent = Agent(
#     system_prompt=SYSTEM_PROMPT,
#     tools=[search_pmc_tool, gather_evidence_tool],
#     model=MODEL_ID,
# )

# # Send a message to the agent
# response = pmc_research_agent(QUERY)

Let's look at the new records in our db table

In [4]:
dynamodb = boto3.resource("dynamodb")

# Check if table exists
table_name = "deep-research-evidence"
table = dynamodb.Table(table_name)

records = [record for record in table.scan().get('Items')]
records

[{'evidence_id': '73c5ecc3-f3d2-4134-bc9a-28d7316ad4ff',
  'question': 'How safe and effective are GLP-1 drugs for long term use?',
  'context': ['In 26‑ to 40‑week trials, tirzepatide (a GIP/GLP‑1 co‑agonist) reduced HbA1c by up to 2.4% and body weight by up to 12.4\u202fkg, outperforming dulaglutide and semaglutide. Gastro‑intestinal adverse events were common (up to 66% GI AEs, 24.5% discontinuations at 15\u202fmg), but cardiovascular safety was favorable across SURPASS studies.',
   'Phase‑3 SURPASS trials of tirzepatide (a GLP‑1/GIP co‑agonist) show sustained HbA1c reduction and weight loss over 52\u202fweeks, with a safety profile comparable to GLP‑1R agonists and no major cardiovascular safety signals, supporting long‑term efficacy and tolerability.',
   'Tirzepatide, a dual GIP/GLP‑1 agonist, shows strong long‑term efficacy: significant HbA1c reductions (up to 2.4%), weight loss (≈3\u202fkg in 29\u202fdays) and superior outcomes vs dulaglutide/semaglutide. Cardiovascular safety

We can also query for a records from a specific PMC ID

In [10]:
example_pmc_id = records[0]["pmc_id"]
print(example_pmc_id)

PMC9438179


In [ ]:
import boto3
from boto3.dynamodb.conditions import Key
dynamodb = boto3.resource("dynamodb")
table_name = "deep-research-evidence"
table = dynamodb.Table(table_name)


response = table.query(
    IndexName="pmc_id_index", KeyConditionExpression=Key("pmc_id").eq(example_pmc_id)
)
example_db_records = response.get('Items')
example_db_records

[{'question': 'How safe and effective are GLP-1 drugs for long term use?',
  'evidence_id': '73c5ecc3-f3d2-4134-bc9a-28d7316ad4ff',
  'answer': "Based on the available evidence, tirzepatide, a dual GLP-1/GIP receptor co-agonist, demonstrates favorable long-term safety and efficacy profiles. The Phase-3 SURPASS trials show sustained HbA1c reduction and weight loss over 52 weeks of treatment (Nauck2022 chunk 2). \n\nRegarding safety, tirzepatide exhibits a safety profile comparable to established GLP-1 receptor agonists, with no major cardiovascular safety signals identified during the 52-week study period (Nauck2022 chunk 2). This suggests that the dual receptor mechanism does not introduce additional cardiovascular risks beyond those associated with traditional GLP-1 therapies.\n\nFor efficacy, the sustained reductions in both glycemic control (HbA1c) and body weight over the full 52-week treatment period indicate maintained therapeutic benefits without significant attenuation of effec

Let's incorporate our tool into a new `pmc_research_agent` definition

In [11]:
from strands import Agent
from search_pmc import search_pmc_tool
from gather_evidence_ddb import gather_evidence_tool
import os

MODEL_ID = "global.anthropic.claude-sonnet-4-20250514-v1:0"
QUERY = "How safe and effective are GLP-1 drugs for long term use?"

SYSTEM_PROMPT = """You are a life science research assistant. When given a scientific question, follow this process:

1. Use search_pmc_tool to find highly-cited papers. Search broadly first, then narrow down. Use temporal filters like "last 2 years"[dp] for recent work.
2. Identify the PMC IDs of the most relevant papers, then submit each ID and the query to the gather_evidence_tool.
3. Generate a concise answer to the question based on the most relevant evidence, followed by a list of the associated `evidence_id` values.
"""

os.environ['EVIDENCE_TABLE_NAME'] = table_name

# Initialize your agent
pmc_research_agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    tools=[search_pmc_tool, gather_evidence_tool],
    model=MODEL_ID,
)

# Send a message to the agent
response = pmc_research_agent(QUERY)

I'll help you find information about the safety and effectiveness of GLP-1 drugs for long-term use. Let me search for recent research on this topic.
Tool #1: search_pmc_tool


INFO | search_pmc | Searching PMC for: GLP-1 agonist long term safety effectiveness AND last 3 years[dp]
INFO | search_pmc | Found 100 article pmcids
INFO | search_pmc | Fetching 100 PM articles
INFO | search_pmc | Successfully fetched 100 articles
INFO | search_pmc | Calculating citation relationships and ranking articles
INFO | search_pmc | Citation ranking completed successfully
INFO | search_pmc | Applied max_filtered_result_count limit: returning 10 of 100 articles
INFO | search_pmc | Top 3 results:
--------------------------------------------------
Title: Glucagon-like peptide-1 receptor: mechanisms and advances in therapy
PMID: 39289339
PMC: PMC11408715
DOI: 10.1038/s41392-024-01931-z
Source: https://doi.org/10.1038/s41392-024-01931-z
Authors: Zhikai Zheng, Yao Zong, Yiyang Ma, Yucheng Tian, Yidan Pang, Changqing Zhang, Junjie Gao
Journal: Signal Transduction and Targeted Therapy
Year: 2024
Abstract: The glucagon-like peptide-1 (GLP-1) receptor, known as GLP-1R, is a vital compo

Now let me search for more specific information about long-term safety studies:
Tool #2: search_pmc_tool


INFO | search_pmc | Searching PMC for: "GLP-1 receptor agonist" "long term safety" "adverse events" AND last 3 years[dp]
INFO | search_pmc | Found 100 article pmcids
INFO | search_pmc | Fetching 100 PM articles
INFO | search_pmc | Successfully fetched 100 articles
INFO | search_pmc | Calculating citation relationships and ranking articles
INFO | search_pmc | Citation ranking completed successfully
INFO | search_pmc | Applied max_filtered_result_count limit: returning 10 of 100 articles
INFO | search_pmc | Top 3 results:
--------------------------------------------------
Title: Emerging Role of GLP-1 Agonists in Obesity: A Comprehensive Review of Randomised Controlled Trials
PMID: 37445623
PMC: PMC10341852
DOI: 10.3390/ijms241310449
Source: https://doi.org/10.3390/ijms241310449
Authors: Mihaela-Simona Popoviciu, Lorena Păduraru, Galal Yahya, Kamel Metwally, Simona Cavalu
Journal: International Journal of Molecular Sciences
Year: 2023
Abstract: Obesity is a chronic disease with high prev

Let me gather detailed evidence from the most relevant articles. I'll start with the comprehensive review articles that specifically address safety and effectiveness:
Tool #3: gather_evidence_tool


...


Tool #6: gather_evidence_tool


INFO | gather_evidence | Starting gather_evidence for PMCID: PMC11786232, question: What are the discontinuation rates, tolerability issues, and patient adherence patterns with long-term GLP-1 agonist use?
INFO | gather_evidence | Attempting to download s3://pmc-oa-opendata/oa_comm/txt/all/PMC11786232.txt to my_papers/PMC11786232/txt/PMC11786232.txt
INFO | gather_evidence | Successfully downloaded file to my_papers/PMC11786232/txt/PMC11786232.txt
INFO | gather_evidence | Successfully retrieved commercial article PMC11786232
INFO | gather_evidence | Processing paper with paper-qa for question: What are the discontinuation rates, tolerability issues, and patient adherence patterns with long-term GLP-1 agonist use?
INFO | gather_evidence | Invoking paper-qa
INFO | gather_evidence | Successfully answered question for PMC11786232
INFO | gather_evidence | Saving record to deep-research-evidence
INFO | gather_evidence | {'evidence_id': '9de02b4d-e72c-4748-ac47-af7731813c1f', 'question': 'What


Tool #7: gather_evidence_tool


INFO | gather_evidence | Starting gather_evidence for PMCID: PMC11751538, question: What are the recent findings regarding suicide risk and psychiatric effects of GLP-1 agonists?
INFO | gather_evidence | Attempting to download s3://pmc-oa-opendata/oa_comm/txt/all/PMC11751538.txt to my_papers/PMC11751538/txt/PMC11751538.txt
INFO | gather_evidence | Successfully downloaded file to my_papers/PMC11751538/txt/PMC11751538.txt
INFO | gather_evidence | Successfully retrieved commercial article PMC11751538
INFO | gather_evidence | Processing paper with paper-qa for question: What are the recent findings regarding suicide risk and psychiatric effects of GLP-1 agonists?
INFO | gather_evidence | Invoking paper-qa
INFO | gather_evidence | Successfully answered question for PMC11751538
INFO | gather_evidence | Saving record to deep-research-evidence
INFO | gather_evidence | {'evidence_id': '04f59e5e-8548-41f2-9bf6-6b930627b31a', 'question': 'What are the recent findings regarding suicide risk and ps

Based on the comprehensive review of recent research literature, here's what we know about the safety and effectiveness of GLP-1 drugs for long-term use:

## Safety Profile for Long-Term Use

**Common Adverse Effects:**
- **Gastrointestinal effects** are the most frequent side effects, including nausea, vomiting, diarrhea, and dyspepsia. These typically occur early in treatment but can persist with chronic use and are dose-dependent.
- **Discontinuation rates** are substantial: 46.5% of patients with diabetes and 64.8% of patients without diabetes discontinue within one year, with GI adverse events being a primary driver (hazard ratio 1.2-1.4).

**Serious but Rare Concerns:**
- **Acute pancreatitis** - rare but serious, particularly in patients with prior history
- **Gallbladder disease** (cholelithiasis, cholecystitis) - modest increased incidence, likely secondary to rapid weight loss
- **Thyroid C-cell effects** - based on preclinical rodent data, FDA contraindications exist for pat

Now let's view the records in our evidence table again

In [12]:
dynamodb = boto3.resource("dynamodb")

table_name = "deep-research-evidence"
table = dynamodb.Table(table_name)

records = [record for record in table.scan().get('Items')]
records

[{'evidence_id': 'bbf4b67c-37aa-4822-9982-d77b3463ef6c',
  'question': 'What are the long-term safety concerns and adverse effects of GLP-1 receptor agonists? What evidence exists for their long-term effectiveness?',
  'context': ['Long‑term liraglutide (highest obesity dose) raises reward‑related orbitofrontal activation to food cues, hinting at weight‑loss plateau as an adverse effect. GLP‑1 agonists show sustained glycaemic and weight benefits in chronic use, and central GLP‑1 activation reduces inflammation, supporting safety.',
   'The excerpt does not detail specific long‑term safety concerns, but notes formulation challenges (e.g., injection resistance, low oral bioavailability) and side‑effect mitigation via combination therapy. It cites evidence of sustained efficacy across multiple indications—obesity, T2DM, cardiovascular risk, CKD, NAFLD, and AD—supporting long‑term effectiveness.',
   '**Long‑term safety concerns & adverse effects of GLP‑1 receptor agonists (GLP‑1RAs)**  \

We'll use our `pmc_research_agent` again later in this notebook

## 3. Define Technical Writing Agent

Next, we'll build our writing agent. This agent will take in a question and citations and use the Anthropic Claude citations API to add citations. Let's test how this works with the Amazon Bedrock InvokeModel API 

Ref: https://docs.claude.com/en/docs/build-with-claude/citations

### 3.1. Explore Anthropic Claude Citations API

Here is an example of how to use the Claude citations API through a Bedrock InvokeModel call

In [ ]:
import boto3
import json

bedrock_client = boto3.client('bedrock-runtime')

example_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "document",
                    "source": {
                        "type": "text",
                        "media_type": "text/plain",
                        "data": "Based on the available context, specific recommendations for managing gastrointestinal adverse events during GLP-1 receptor agonist (GLP-1 RA) therapy include several key strategies. Management emphasizes comprehensive patient education, initiating treatment with low starting doses, and implementing gradual dose titration to minimize adverse events (Gorgojo2023 chunk 2). Dietary adjustments and ongoing monitoring are also essential components of the management approach (Gorgojo2023 chunk 2).\n\nRegarding clinical experience with long-term safety, GI adverse events occur in 40-85% of patients receiving GLP-1 RAs, but these events are typically mild and transient in nature (Gorgojo2023 chunk 2). The adverse events generally resolve following dose escalation, indicating that patients can adapt to therapy over time (Gorgojo2023 chunk 2). \n\nLong-term safety data demonstrate a favorable profile, with most events classified as non-serious and low rates of permanent treatment discontinuation (Gorgojo2023 chunk 2). This clinical experience supports the overall safety profile of GLP-1 RAs for long-term use (Gorgojo2023 chunk 2). The combination of appropriate management strategies and the generally mild, self-limiting nature of GI adverse events allows most patients to continue therapy successfully over extended periods.",
                    },
                    "title": "PMC9821052",
                    "citations": {"enabled": True},
                },
                {
                    "type": "document",
                    "source": {
                        "type": "text",
                        "media_type": "text/plain",
                        "data": "Long-term GLP-1 receptor agonist therapy demonstrates sustained efficacy in maintaining glycemic control and promoting weight loss (Zheng2024 chunk 55). However, therapeutic effectiveness may plateau over extended treatment periods, with studies indicating this limitation is associated with increased orbitofrontal reward activation, as observed with liraglutide (Zheng2024 chunk 55).\n\nThe adverse event profile is predominantly gastrointestinal, with nausea and vomiting representing the most common side effects (Zheng2024 chunk 55). Serious but rare adverse events include pancreatitis and gallbladder disease (Zheng2024 chunk 55).\n\nSeveral absolute contraindications exist for long-term GLP-1 receptor agonist use. Patients with a personal or family history of medullary thyroid carcinoma should not receive these agents (Zheng2024 chunk 55). Multiple endocrine neoplasia type 2 (MEN 2) also represents a contraindication (Zheng2024 chunk 55). Additionally, severe gastrointestinal disorders preclude the use of these medications (Zheng2024 chunk 55).\n\nThe safety and efficacy profile supports long-term use in appropriate patients, though careful patient selection is essential given the specific contraindications, particularly those related to thyroid malignancy risk and pre-existing gastrointestinal pathology.",
                    },
                    "title": "PMC9821052",
                    "citations": {"enabled": True},
                },                
                {
                    "type": "text",
                    "text": "What is the long-term safety profile and effectiveness of GLP-1 receptor agonists? What are the main adverse events and contraindications for long-term use?",
                },
            ],
        }
    ],
    "max_tokens": 1024,
}

response = bedrock_client.invoke_model(
    modelId="us.anthropic.claude-sonnet-4-20250514-v1:0",
    contentType="application/json",
    accept="application/json",
    body=json.dumps(example_request),
)

response_body = json.loads(response["body"].read())

print(response_body.get('content'))

We can now format the response into a citated output

In [ ]:
def print_cited_response(response_content: dict) -> None:
    citations = []
    for content_item in response_content:
        print(content_item.get("text"), end="")
        for citation in content_item.get("citations", []):
            title = citation.get("document_title")
            if title not in citations:
                citations.append(title)
            print(f" ({citations.index(title)+1})", end="")

    print("\n")
    print("## References")
    for i, title in enumerate(citations, start=1):
        print(f"{i}. https://www.ncbi.nlm.nih.gov/pmc/articles/{title}")
    return None

print_cited_response(response_body.get('content'))

Let's try it with an example record from our db. In this case, we're using a custom content type to pass all of the gathered evidence as a single, citable block. We also include the generated answer as additional context (this won't be cited)

In [ ]:
# Defined above
example_db_records 

In [ ]:
import boto3
import json

def parse_db_records(records):
    """ Parse records from our DynamoDB table into content blocks for the Anthropic Claude citation API"""

    contents = []
    for record in records:
        contents.append(
            {
                "type": "document",
                "source": {
                    "type": "content",
                    "content": [
                        # In this case, we join all of the evidence items together.
                        {"type": "text", "text": "".join(record.get("evidence"))}
                        # If we wanted to reference individual chunks, we could also do something like:
                        # {"type": "text", "text": evidence} for evidence in record.get("evidence")
                    ],
                },
                "title": record.get("pmcid"),
                "context": record.get("answer"),
                "citations": {"enabled": True},
            },
        )

    contents.append(
        {
            "type": "text",
            "text": "What is the long-term safety profile and effectiveness of GLP-1 receptor agonists? What are the main adverse events and contraindications for long-term use?",
        },
    )
    return(contents)


In [ ]:
contents = parse_db_records(example_db_records)

print("Bedrock InvokeModel request contents are:\n")
for i in contents:
    print(i)
print()

bedrock_client = boto3.client("bedrock-runtime")

example_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [{"role": "user", "content": contents}],
    "max_tokens": 1024,
}

response = bedrock_client.invoke_model(
    modelId="us.anthropic.claude-sonnet-4-20250514-v1:0",
    contentType="application/json",
    accept="application/json",
    body=json.dumps(example_request),
)

response_body = json.loads(response["body"].read())

print_cited_response(response_body.get('content'))

Let's try one more time, this time with a system prompt with the writing guidelines we gave to our agent in notebook 3

In [ ]:
from datetime import date

SYSTEM_PROMPT = f"""
The current date is {date.today().strftime('%B %d, %Y')}

You are an expert technical writer that answers biomedical questions using scientific literature and other authoritative sources. 
You maintain user trust by being consistent (dependable or reliable), benevolent (demonstrating good intent, connectedness, and care), transparent (truthful, humble, believable, and open), and competent (capable of answering questions with knowledge and authority).
Use a professional tone that prioritizes clarity, without being overly formal.
Use precise language to describe technical concepts. For example, use, "femur" instead of "leg bone" and "cytotoxic T lymphocyte" instead of "killer T cell".

Structure your output as a comprehensive document that clearly communicates your research findings to the reader. Follow these guidelines:

Report Structure:

- Begin with a concise introduction (1-2 paragraphs) that establishes the research question, explains why it's important, and provides a brief overview of your approach
- Organize the main body into sections that correspond to the major research tasks you completed (e.g., "Literature Review," "Current State Analysis," "Comparative Assessment," "Technical Evaluation," etc.)
- Conclude with a summary section (1-2 paragraphs) that synthesizes key findings and discusses implications

Section Format:

- Write each section in paragraph format using 1-3 well-developed paragraphs
- Each paragraph should focus on a coherent theme or finding
- Use clear topic sentences and logical flow between paragraphs
- Integrate information from multiple sources within paragraphs rather than listing findings separately

Citation Requirements:

- Include proper citations for all factual claims using the format provided in your source materials
- Place citations at the end of sentences before punctuation (e.g., "Recent studies show significant progress in this area .")
- Group related information from the same source under single citations when possible
- Ensure every major claim is supported by appropriate source attribution

Writing Style:

- Use clear, professional academic language appropriate for scientific communication
- Use active voice and strong verbs
- Synthesize information rather than simply summarizing individual sources
- Draw connections between different pieces of information and highlight patterns or contradictions
- Focus on analysis and interpretation, not just information presentation
- Don't use unnecessary words. Keep sentences short and concise.
- WRite for a global audience. Avoid jargon an colloquial language. 

Quality Standards:

- Ensure logical flow between sections and paragraphs
- Maintain consistency in terminology and concepts throughout
- Provide sufficient detail to support conclusions while remaining concise
- End with actionable insights or clear implications based on your research findings
"""


In [ ]:
bedrock_client = boto3.client("bedrock-runtime")

example_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [{"role": "user", "content": parse_db_records(example_db_records)}],
    "max_tokens": 2048,
    "system": SYSTEM_PROMPT,
}

response = bedrock_client.invoke_model(
    modelId="us.anthropic.claude-sonnet-4-20250514-v1:0",
    contentType="application/json",
    accept="application/json",
    body=json.dumps(example_request),
)

response_body = json.loads(response["body"].read())

print_cited_response(response_body.get("content"))

### 3.2. Create citation tool